# 토픽분석
    - description 컬럼 추출
     - 명사토큰추출 okt.nouns()

In [2]:
url = 'https://raw.githubusercontent.com/leekyuyoung20221226/python/main/data/%EC%BD%94%EB%A1%9C%EB%82%98%20%EB%89%B4%EC%8A%A4.csv'
import pandas as pd
import numpy as np

In [3]:
data_df = pd.read_csv(url)
description = data_df['description']

In [5]:
from konlpy.tag import Okt

In [7]:
okt = Okt()
description_noun_tk = []
for d in description:
    description_noun_tk.append(okt.nouns(d)) # 명사 형태소만 추출
# 에러  : phrase input should be string, not <class 'float'>

AssertionError: phrase input should be string, not <class 'float'>

In [27]:
# NaN은 파이썬에서 float 타입이다. 그래서 위에서 전처리를 안해서 에러가 발생했음
type(data_df[data_df['description'].isnull()]['description'].values[0])

float

In [28]:
data_df.dropna(inplace=True)

In [31]:
description_noun_tk = []
description = data_df['description']
for d in description:
    description_noun_tk.append(okt.nouns(d)) # 명사 형태소만 추출

In [48]:
#  토큰의 길이가 1보다 큰 것만 추출
description_noun_tk2 = []
for d in description_noun_tk:
     description_noun_tk2.append([i for i in d if len(i) > 1])

In [50]:
len(description_noun_tk2)

1091

# 토픽분석을 위한 LDA 모델 구축
    gensim 라이브러리 추가

In [38]:
!pip install gensim

     --------------------------------------- 24.0/24.0 MB 50.4 MB/s eta 0:00:00
     ---------------------------------------- 56.8/56.8 kB ? eta 0:00:00
     ------------------------------------- 986.3/986.3 kB 30.5 MB/s eta 0:00:00
     ---------------------------------------- 67.1/67.1 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20431 sha256=fd01166ae2d0a34f3493d918ad8e48a840fec3577297c7c31daf2da4efb6f70b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\01\02\ee\df0699282986903a384b69aab4413af9efd26b3612b5dccc9e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3514 sha256=124d749418e605cbc28e047c6b6be8f457b3c6e7b00ad968a3067bc03c387a6f
  Stored in directory: c:\users\user\appdata\local\pip\cach

In [40]:
import gensim
import gensim.corpora as corpora

In [51]:
### LDA 토픽모델의 입력 베터 생성
dictionary =  corpora.Dictionary(description_noun_tk2)  # 사전 구성

In [52]:
dictionary[0]  # 확인

'감염증'

In [59]:
# 단어 뭉치를 구성
corpus =  [dictionary.doc2bow(word) for word in description_noun_tk2]
# print(corpus)

In [90]:
# 토픽의 개수를 설정 - 하이퍼 파라메터
k = 2
lda_model =  gensim.models.ldamulticore.LdaMulticore(corpus,iterations=12,num_topics=k, id2word = dictionary)

In [91]:
# help(gensim.models.ldamulticore.LdaMulticore)

In [92]:
# 토픽분석 결과 출력
print(lda_model.print_topics(num_topics=k, num_words=15))
# 토픽을 구성하는 주요  단어 15개가 토픽에 대한 영향력 비율

[(0, '0.093*"코로나" + 0.021*"진자" + 0.013*"신규" + 0.010*"접종" + 0.010*"방역" + 0.009*"중국" + 0.008*"백신" + 0.007*"중앙" + 0.007*"발생" + 0.007*"대책" + 0.007*"기준" + 0.006*"본부" + 0.006*"검사" + 0.006*"이후" + 0.005*"지난"'), (1, '0.075*"코로나" + 0.021*"진자" + 0.011*"중국" + 0.011*"방역" + 0.010*"신규" + 0.010*"발생" + 0.009*"접종" + 0.008*"검사" + 0.008*"백신" + 0.007*"대책" + 0.007*"기준" + 0.007*"본부" + 0.007*"이후" + 0.006*"재난" + 0.006*"지난"')]


In [93]:
lda_topics = lda_model.print_topics(num_topics=k, num_words=15)

In [94]:
len(lda_topics), len(lda_topics[0][1].split('+'))

(2, 15)

In [95]:
#  d = {'col1': [1, 2], 'col2': [3, 4]}

In [96]:
dic1 = {}
dic1['토픽번호'] = [i[0] for i in lda_topics]
dic1['주요단어(15)'] = [i[1] for i in lda_topics]

In [97]:
pd.options.display.max_colwidth = None

In [98]:
pd.DataFrame(dic1)

,토픽번호,주요단어(15)
0,0,"0.093*""코로나"" + 0.021*""진자"" + 0.013*""신규"" + 0.010*""접종"" + 0.010*""방역"" + 0.009*""중국"" + 0.008*""백신"" + 0.007*""중앙"" + 0.007*""발생"" + 0.007*""대책"" + 0.007*""기준"" + 0.006*""본부"" + 0.006*""검사"" + 0.006*""이후"" + 0.005*""지난"""
1,1,"0.075*""코로나"" + 0.021*""진자"" + 0.011*""중국"" + 0.011*""방역"" + 0.010*""신규"" + 0.010*""발생"" + 0.009*""접종"" + 0.008*""검사"" + 0.008*""백신"" + 0.007*""대책"" + 0.007*""기준"" + 0.007*""본부"" + 0.007*""이후"" + 0.006*""재난"" + 0.006*""지난"""


In [99]:
dic1['토픽레이블'] = ['확진자 신규 접종 대책','확진자 중국 방역 대책']

In [101]:
pd.DataFrame(dic1)

,토픽번호,주요단어(15),토픽레이블
0,0,"0.093*""코로나"" + 0.021*""진자"" + 0.013*""신규"" + 0.010*""접종"" + 0.010*""방역"" + 0.009*""중국"" + 0.008*""백신"" + 0.007*""중앙"" + 0.007*""발생"" + 0.007*""대책"" + 0.007*""기준"" + 0.006*""본부"" + 0.006*""검사"" + 0.006*""이후"" + 0.005*""지난""",확진자 신규 접종 대책
1,1,"0.075*""코로나"" + 0.021*""진자"" + 0.011*""중국"" + 0.011*""방역"" + 0.010*""신규"" + 0.010*""발생"" + 0.009*""접종"" + 0.008*""검사"" + 0.008*""백신"" + 0.007*""대책"" + 0.007*""기준"" + 0.007*""본부"" + 0.007*""이후"" + 0.006*""재난"" + 0.006*""지난""",확진자 중국 방역 대책


# 분석결과 시각화